In [1]:
from pyomo.environ import *
import pandas as pd
import plotly.graph_objects as go

from math import comb
import re

In [2]:
## data 

data = pd.DataFrame({
    'Team' : ['Atlanta','Philly','New York','Montreal'],
    'Wins' : [83,80,78,77],
    'Atlanta' : ['-',1,6,1],
    'Philly' : [1,'-',0,2],
    'New York' : [6,0,'-',0],
    'Montreal' : [1,2,0,'-']
})

remaining = []

for indx in range(data.shape[0]):
    sum = 0
    for col in data.columns[2:]:
        val = data.loc[indx,col]
        if val != '-':
            sum += val
    remaining.append(sum)

data['Rem'] = remaining

data

,Team,Wins,Atlanta,Philly,New York,Montreal,Rem
0,Atlanta,83,-,1,6,1,8
1,Philly,80,1,-,0,2,3
2,New York,78,6,0,-,0,6
3,Montreal,77,1,2,0,-,3


In [3]:
analyseTeam = 'Atlanta'
indx = list(data['Team']).index(analyseTeam)

UBwins = data.loc[indx,'Rem'] + data.loc[indx,'Wins']

print(f"Analyzing tame {analyseTeam} -> curr score {data.loc[indx,'Wins']}, matches remaining {data.loc[indx,'Rem']} --> UB on points = {UBwins}")

data.drop(indx, inplace=True)
data.drop([analyseTeam,'Rem'],axis=1, inplace=True)

data.reset_index(inplace=True, drop=True)

data

Analyzing tame Atlanta -> curr score 83, matches remaining 8 --> UB on points = 91


,Team,Wins,Philly,New York,Montreal
0,Philly,80,-,0,2
1,New York,78,0,-,0
2,Montreal,77,2,0,-


In [4]:
# Graph

Graph = dict()

# Source to matches  (weight -> matches remaining - r_ij)

Graph.update({'s':[]})
for t1 in range(data.shape[0]):
    team1 = data.loc[t1,'Team']
    for t2 in range(t1+1,data.shape[0]):
         team2 = data.loc[t2,'Team']
         Graph['s'].append((f'{team1}-{team2}',data.loc[t1,team2]))

# matches to teams

for match in Graph['s']:
    Graph.update({match[0]:match[0].split('-')})

# teams to sink

for indx,team in enumerate(data['Team']):
    Graph.update({team:('t',UBwins-data.loc[indx,'Wins'])})

Graph

{'s': [('Philly-New York', 0),
  ('Philly-Montreal', 2),
  ('New York-Montreal', 0)],
 'Philly-New York': ['Philly', 'New York'],
 'Philly-Montreal': ['Philly', 'Montreal'],
 'New York-Montreal': ['New York', 'Montreal'],
 'Philly': ('t', 11),
 'New York': ('t', 13),
 'Montreal': ('t', 14)}

In [221]:
model = ConcreteModel()

src_match_vars = []
for matches in Graph['s']:
    model.add_component(f's > {matches[0]}',Var(domain=NonNegativeReals, bounds=(0,matches[1])))
    src_match_vars.append(model.component(f's > {matches[0]}'))

match_team_vars = []
for node in Graph.keys():
    if node.find('-') != -1:
        for team in Graph[node]:
            model.add_component(f'{node} > {team}',Var(domain=NonNegativeReals))
            match_team_vars.append(model.component(f'{node} > {team}'))

team_snk_vars = []
for node in Graph.keys():
    if node != 's' and node.find('-') == -1:
        model.add_component(f'{node} > t',Var(domain=NonNegativeReals, bounds=(0,Graph[node][1])))
        team_snk_vars.append(model.component(f'{node} > t'))

# objective Max flow

sum = 0
for var in src_match_vars:
    sum += var

model.Obj = Objective(expr = sum, sense=maximize)


# flow src-match-team cons

model.SrcMatchTeamCons = ConstraintList()

for src_mch_var in src_match_vars:
    mch = src_mch_var.getname().split(' > ')[1]
    sum = 0
    for mch_team_var in match_team_vars:
        if re.match(mch,mch_team_var.getname()):
            sum += mch_team_var
    model.SrcMatchTeamCons.add(expr = src_mch_var == sum)

# flow match-team-sink cons

model.MatchTeamSinkCons = ConstraintList()

for tm_snk_var in team_snk_vars:
    tm = tm_snk_var.getname().split(' > ')[0]
    sum = 0
    for mth_tm_var in match_team_vars:
        if mth_tm_var.getname().split(' > ')[1] == tm:
            sum += mth_tm_var
    model.MatchTeamSinkCons.add(expr = sum == tm_snk_var)

model.pprint()

2 Set Declarations
    MatchTeamSinkCons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    SrcMatchTeamCons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

12 Var Declarations
    Montreal > t : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :    14 : False :  True : NonNegativeReals
    New York > t : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :    13 : False :  True : NonNegativeReals
    New York-Montreal > Montreal : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    New York-Montreal > New York : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : 

In [222]:
minotaur = SolverFactory("mbnb", executable='../CHL_Grouping/mbin/mbnb')
res = minotaur.solve(model)
res.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 12
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: 
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.005051374435424805
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [223]:
if res.Solver.termination_condition == TerminationCondition.infeasible:
    print('Infeasible')
else:
    print(f'max flow : {model.Obj()}')

sum = 0
for arc in Graph['s']:
    sum += arc[1]

print(f'Bound on source flow {sum}')

if model.Obj() == sum:
    print('Saturated Flow')

max flow : 2.0
Bound on source flow 2
Saturated Flow


In [224]:
N = 6                         # number of teams

fig = go.Figure()

fig.update_layout(
    height = 900,
    width = 1000
)

# source 

fig.add_trace(go.Scatter(x=[1], y=[(comb(N,2)+1)/2], mode='markers' ,marker=dict(size=20, color='gray', line=dict(width = 2)), opacity=0.8, showlegend=False))

# team matches

yval = 1
for t1 in range(N):
    for t2 in range(t1+1,N):
        fig.add_trace(go.Scatter(x=[2],y=[yval], mode='markers' ,marker=dict(size=20, line=dict(width = 2)), opacity=0.5, showlegend=False))
        yval += 1

# teams

yval = (comb(N,2)+1)/2 - (N-1)/2
for t1 in range(N):
    fig.add_trace(go.Scatter(x=[3],y=[yval], mode='markers' ,marker=dict(size=20, line=dict(width = 2)), opacity=0.5, showlegend=False))
    yval += 1


# sink

fig.add_trace(go.Scatter(x=[4], y=[(comb(N,2)+1)/2], mode='markers' ,marker=dict(size=20, color='gray', line=dict(width = 2)), opacity=0.8, showlegend=False))

fig.show()